Notebook for harry testing

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc


db_path = r"C:\Users\Harry Hall\OneDrive - UNSW\Desktop\MASTERS\S1\Research & Innovation\laliga.sqlite"
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM Matches", conn)
conn.close()
display(df.head())

conn.close()

: 

Now need to clean the data and process into usable features.  
  
Firslty, dropping rows with no score and then splitting score into home and away component.  
  
I think it also makes sense to use the final notation suggested in the assignment of: home_win = 1, away_win = 2, tie = X  
  

In [ ]:
df = df.dropna(axis = 'index' , how = 'any', subset = 'score')
df[['home_score', 'away_score']] = df['score'].str.split(':', expand=True)
df['home_score'] = pd.to_numeric(df['home_score'])
df['away_score'] = pd.to_numeric(df['away_score'])
df = df[(~df['home_score'].isnull()) | (~df['away_score'].isnull())]

df['result'] = df.apply(lambda row: '1' if row['home_score'] > row['away_score']
                                        else '2' if row['home_score'] < row['away_score']
                                        else 'X', axis=1)



-goals scores (at home and away)  
-goals conceded (at home and away)  
-goal difference  
-current ranking  
-average ranking in the last 5 seasons for example  
-current streak of games (average number of points gained in the last X games)  
-head2head (goals scored, conceded, wins, ties...)  
-general win/tie/loses rate of each team  
-Number of promotions and relegations ?  
-biggest win ?  
-longest undefeated streak ?  
-day and hour ?  

In [ ]:
df['gd_5_game_home'] = None
df['gd_5_game_away'] = None
df['form_10_game_home'] = None
df['form_10_game_away'] = None

df['live_rank_home'] = None
df['live_rank_away'] = None

def calculate_gd_5_game(df):
    teams = df['home_team'].unique()
    for team in teams:
        team_df = df[(df['home_team'] == team) | (df['away_team'] == team)].copy()
        team_df['goal_difference'] = team_df.apply(
            lambda x: x['home_score'] - x['away_score'] if x['home_team'] == team else x['away_score'] - x['home_score'], axis=1
        )
        team_df['gd_5_game'] = team_df['goal_difference'].rolling(window=5).sum().shift(1)
        
        # Assign gd_5_game to the appropriate column based on home or away
        df.loc[team_df[team_df['home_team'] == team].index, 'gd_5_game_home'] = team_df.loc[team_df['home_team'] == team, 'gd_5_game']
        df.loc[team_df[team_df['away_team'] == team].index, 'gd_5_game_away'] = team_df.loc[team_df['away_team'] == team, 'gd_5_game']
    
    return df

def calculate_form_10_game(df):
    teams = df['home_team'].unique()
    for team in teams:
        team_df = df[(df['home_team'] == team) | (df['away_team'] == team)].copy()
        team_df['win'] = team_df.apply(
            lambda x: 1 if (x['home_team'] == team and x['home_score'] > x['away_score']) or 
                           (x['away_team'] == team and x['away_score'] > x['home_score']) else 0, axis=1
        )
        team_df['form_10_game'] = team_df['win'].rolling(window=10).sum().shift(1)
        
        # Assign form_10_game to the appropriate column based on home or away
        df.loc[team_df[team_df['home_team'] == team].index, 'form_10_game_home'] = team_df.loc[team_df['home_team'] == team, 'form_10_game']
        df.loc[team_df[team_df['away_team'] == team].index, 'form_10_game_away'] = team_df.loc[team_df['away_team'] == team, 'form_10_game']
    
    return df

# THIS FUNCTION NEEDS WORK
def calculate_live_rank(df):
    teams = df['home_team'].unique()
    points_table = {team: 0 for team in teams}
    matchdays = sorted(df['matchday'].unique())
    
    for matchday in matchdays:
        day_df = df[df['matchday'] == matchday]
        for _, row in day_df.iterrows():
            home_team, away_team = row['home_team'], row['away_team']
            home_score, away_score = row['home_score'], row['away_score']
            
            # Update points based on match result
            if home_score > away_score:
                points_table[home_team] += 3
            elif away_score > home_score:
                points_table[away_team] += 3
            else:
                points_table[home_team] += 1
                points_table[away_team] += 1
        
        # Sort teams by points and assign ranks
        sorted_teams = sorted(points_table.items(), key=lambda x: x[1], reverse=True)
        rank_dict = {team: rank+1 for rank, (team, _) in enumerate(sorted_teams)}
        
        # Update live rank for this matchday for both home and away teams
        for _, row in day_df.iterrows():
            df.loc[row.name, 'live_rank_home'] = rank_dict[row['home_team']]
            df.loc[row.name, 'live_rank_away'] = rank_dict[row['away_team']]
    
    return df


df = calculate_gd_5_game(df)
df = calculate_form_10_game(df)
df = calculate_live_rank(df)


display(df.head())
display(df.tail())


In [ ]:
df['prev_season_rank_home'] = None
df['prev_season_rank_away'] = None

def calculate_season_ranks(df):

    prev_season_ranks = {}

    seasons = df['season'].unique()
    for season in seasons:

        season_df = df[df['season'] == season].copy()

        points_table = {}

        for _, row in season_df.iterrows():
            home_team, away_team = row['home_team'], row['away_team']
            home_score, away_score = row['home_score'], row['away_score']

            if home_team not in points_table:
                points_table[home_team] = 0
            if away_team not in points_table:
                points_table[away_team] = 0

            if home_score > away_score:
                points_table[home_team] += 3
            elif away_score > home_score:
                points_table[away_team] += 3
            else:
                points_table[home_team] += 1
                points_table[away_team] += 1

        ranked_teams = sorted(points_table.items(), key=lambda x: x[1], reverse=True)
        season_ranking = {team: rank+1 for rank, (team, _) in enumerate(ranked_teams)}

        for team, rank in season_ranking.items():
            prev_season_ranks[(season, team)] = rank
    
    return prev_season_ranks

prev_season_ranks = calculate_season_ranks(df)


def assign_prev_season_rank(df, prev_season_ranks):

    for i, row in df.iterrows():
        current_season = row['season']
        home_team = row['home_team']
        away_team = row['away_team']

        prev_season_year = str(int(current_season.split('-')[0]) - 1) + '-' + str(int(current_season.split('-')[1]) - 1)

        df.at[i, 'prev_season_rank_home'] = prev_season_ranks.get((prev_season_year, home_team), None)
        df.at[i, 'prev_season_rank_away'] = prev_season_ranks.get((prev_season_year, away_team), None)
    
    return df

df = assign_prev_season_rank(df, prev_season_ranks)

display(df[['season', 'home_team', 'away_team', 'prev_season_rank_home', 'prev_season_rank_away']].head())
display(df.tail())


: 

In [41]:
y = df.loc[:,'result']

cols_to_drop = ['score','home_score','away_score','result']
X = df.drop(columns=cols_to_drop)
X = X.drop(columns=['date', 'time'])
print(X.columns)
print(y[:10,])

X.corr(numeric_only=True).style.background_gradient("coolwarm", vmin=-1, vmax=1)

Index(['season', 'division', 'matchday', 'home_team', 'away_team',
       'gd_5_game_home', 'gd_5_game_away', 'form_10_game_home',
       'form_10_game_away', 'live_rank_home', 'live_rank_away',
       'prev_season_rank_home', 'prev_season_rank_away'],
      dtype='object')
0    2
1    1
2    1
3    X
4    2
5    2
6    1
7    2
8    1
9    1
Name: result, dtype: object


TypeError: corr() got an unexpected keyword argument 'numeric_only'